### Generate data into Event Hubs

In [2]:
tps = 10
# make sure to use your Event Hubs connection string to the event hubs created (not the namespace level)
eh_connection = "put your event hubs connection here"   # or use dbutils.secrets.get(scope="your scope",key="your scope key to the eventhubs connection")

In [3]:
rateStream = (spark.readStream
                   .format("rate")
                   .option("rowsPerSecond", tps)
                   .option("numPartitions", 1)
                   .load())

In [4]:
genDataStream = (rateStream
                 .selectExpr("value as messageId",
                             "cast(rand(value)*"+str(tps*10)+" as integer) as deviceId",
                             "cast(rand(value)*20 as integer)+20 as temperature",
                             "timestamp as genTimestamp")
                 .selectExpr("to_json(struct(*)) as body"))

In [5]:
# path for checkpoints
checkpointPath = '/checkpoint/seedEventHubs'
dbutils.fs.rm(checkpointPath, recurse=True)

# connection info
ehConf = {'eventhubs.connectionString' : eh_connection}

query = (genDataStream
    .writeStream
    .format("eventhubs")
    .outputMode("update")
    .options(**ehConf)
    .trigger(processingTime='1 seconds')
    .option("checkpointLocation", checkpointPath)
    .start())